In [3]:
# OS/ User specific 

TRAINING_DATASET_PATH = "/home/jcharlet/workspace/datascience/machine_learning_notes/data" + '/train.csv'
# TRAINING_DATASET_PATH = os.path.abspath(os.path.join( os.getcwd() , 'data')) + r'\train.csv'

### Kaggle project - Table of Contents
* 1 - Define the problem
* [Load data and displaying info](#load-data)
* 2 - Prepare Data
    * Identify features
        * Separate numerical from categorical features
        * Separate nominal and ordinal (from categorical features)
    * Clean data
        * Remove numerical features with missing values
        * Remove categorical features with missing values
        * drop outliers in numerical values # WIP
    * transform # TODO
        * transform categorical values #TODO
* 2 - Feature selection #WIP
    * Select features using random forest classifier #WIP
    
* 3 - Spot Check Algorithms
    * split dataset
    * train on multiple algorithms

# Define the problem

From project https://www.kaggle.com/c/house-prices-advanced-regression-techniques

- Step 1: What is the problem? 

house prices are hard to predict, we want to know how much my beautiful flat is worth on the market in the USA

- Step 2: Why does the problem need to be solved? 

MONEY MONEY
and especially we need to learn Machine Learning

- Step 3: How would I solve the problem? 

housing agents do statistical studies.

Data pre-processing is very important. We have a lot of features, dirty data (missing values), numerical/ordinal/nominal values.

We'll follow the process from https://machinelearningmastery.com/process-for-working-through-machine-learning-problems/

- 1 - Define the Problem
- 2 - Prepare Data
- 3 - Spot Check Algorithms
- 4 - Improve Results
- 5 - Present Results

# Load data <a class="anchor" id="load-data"></a>

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv(TRAINING_DATASET_PATH)


In [6]:
# print(df.info())


# Prepare data <a class="anchor" id="clean-data"></a>
## Identify features
### Separate numerical from categorical features

In [7]:
# Separating data features according to data type: numeric or string

X_numeric_labels = set(df._get_numeric_data().columns.tolist())
X_categorical_labels = set(df.columns.tolist()).difference(X_numeric_labels)

In [8]:
print('These are numerical features {} \n'.format(X_numeric_labels))
print('These are categorical features {} \n'.format(X_categorical_labels))


These are numerical features {'YearBuilt', 'BsmtFinSF2', 'SalePrice', 'LowQualFinSF', 'FullBath', 'TotRmsAbvGrd', 'GarageYrBlt', 'KitchenAbvGr', 'MoSold', 'GarageCars', 'WoodDeckSF', 'LotArea', 'BsmtUnfSF', 'BsmtFullBath', 'MasVnrArea', 'TotalBsmtSF', 'OpenPorchSF', '1stFlrSF', 'ScreenPorch', '2ndFlrSF', 'MSSubClass', 'Id', 'MiscVal', 'PoolArea', 'BsmtFinSF1', 'HalfBath', 'OverallQual', 'Fireplaces', 'BsmtHalfBath', 'BedroomAbvGr', 'YearRemodAdd', 'GarageArea', '3SsnPorch', 'EnclosedPorch', 'OverallCond', 'LotFrontage', 'GrLivArea', 'YrSold'} 

These are categorical features {'ExterCond', 'RoofStyle', 'Functional', 'Electrical', 'CentralAir', 'Foundation', 'BsmtQual', 'GarageQual', 'SaleCondition', 'MSZoning', 'Alley', 'ExterQual', 'GarageCond', 'FireplaceQu', 'GarageFinish', 'Exterior2nd', 'PoolQC', 'LotShape', 'LandContour', 'LandSlope', 'KitchenQual', 'SaleType', 'MiscFeature', 'BsmtCond', 'GarageType', 'Street', 'PavedDrive', 'LotConfig', 'Exterior1st', 'BsmtExposure', 'BsmtFinType

### Separate nominal and ordinal (from categorical features)

In [9]:
# Separating categorical features in two sets: nominal and ordinal
# The values in features asociated with categorical ordinal feautures are: [Ex, Gd, GLQ, GdPrv, Fin] ... etc

categorical_ordinal_list_values = ['Ex', 'Gd', 'GLQ', 'GdPrv', 'Fin'] # There are not iall of the values. Here must be
# included all the possible values for the categorical ordinal features

X_categorical_nominal_labels = (df[list(X_categorical_labels)].drop(['GarageCond', 'BsmtFinType1', 'BsmtQual','MiscFeature','Alley','GarageType','BsmtFinType2',
         'BsmtExposure', 'FireplaceQu', 'MasVnrType', 'GarageQual', 'GarageFinish', 'Fence', 'PoolQC'],
        1).isin(categorical_ordinal_list_values) == False).all().loc[lambda df: df.values == True].axes

# The previous instruction returns the index of the categorical_nominal_values. I think we could include the following features:
#LandSlope has values: Gtl -> Gentle slopen; Mod -> Moderate Slope; Sev -> Severe Slope
#Functional has values: Typ -> Typical Functionality; Min1 -> Minor Deductions 1; Min2 -> Minor Deductions 2; Mod -> Moderate Deductions; Maj1 -> Major Deductions 1; Maj2 -> Major Deductions 2; Sev -> Severely Damaged; Sal -> Salvage only
#PavedDrive has values:   Y -> Paved ; P -> Partial Pavement; N -> Dirt/Gravel



In [10]:
print(X_categorical_nominal_labels)
print()
print(X_categorical_nominal_labels[0].tolist())

[Index(['RoofStyle', 'Functional', 'Electrical', 'CentralAir', 'Foundation',
       'SaleCondition', 'MSZoning', 'Exterior2nd', 'LotShape', 'LandContour',
       'LandSlope', 'SaleType', 'Street', 'PavedDrive', 'LotConfig',
       'Exterior1st', 'HouseStyle', 'RoofMatl', 'Utilities', 'Condition1',
       'Heating', 'Condition2', 'BldgType', 'Neighborhood'],
      dtype='object')]

['RoofStyle', 'Functional', 'Electrical', 'CentralAir', 'Foundation', 'SaleCondition', 'MSZoning', 'Exterior2nd', 'LotShape', 'LandContour', 'LandSlope', 'SaleType', 'Street', 'PavedDrive', 'LotConfig', 'Exterior1st', 'HouseStyle', 'RoofMatl', 'Utilities', 'Condition1', 'Heating', 'Condition2', 'BldgType', 'Neighborhood']


### Remarks
<p> From the set of set of **X_categorical_labels** the set of **X_categorical_nominal_labels** is extracted to get 
the set of **X_categorical_ordinal_labels**</p>

* On the set of **X_categorical_ordinal_labels** we can apply pag 104 **mapping strategy**

* On the set of **X_categorical_nominal_labels** we can apply **one-hot encoding strategy** pag 106

In [11]:
X_categorical_ordinal_labels = set(X_categorical_labels).difference(set(X_categorical_nominal_labels[0].tolist()))

In [12]:
# print(X_categorical_labels)

In [13]:
# df[list(X_categorical_ordinal_labels)]

## Clean data
### Remove numerical features with missing values

#### Some considerations


* <p> Page 107 of "our book":  "After executing the preceding code, the first column of the NumPy array X now holds the new
colour values, which are encoded as follows:
blue to 0
green to 1
red to 2
If we stop at this point and feed the array to our classifier, *we will make one of the most common 
mistakes in dealing with categorical data*. Can you spot the problem? *
Although the colour values do not come in any particular order, a learning algorithm will now assume
that green is larger than blue, and red is larger than green. 
Although this assumption is incorrect, the algorithm could still produce useful results. 
However, those results would not be optimal* "</p>

* <p> In the numerical feature variables there is the possibility that there are categorical numerical features variables. How to identify this kinf of features ? 
 I think the main problem is what strategy to use to replace values ? median, mode or mean ? </p>
 

In [14]:
df_cleaned=df.copy()


#Let's check how many numerical features have missing values

# print(df[df._get_numeric_data().columns.tolist()].isnull().sum())

# Dropping columns: ['Id', 'LotFrontage', 'GarageYrBlt', 'SalePrice',] from numerical features

# dropping Id and 
df_cleaned= df_cleaned._get_numeric_data().drop(['Id'],1)
X_numerical_not_missing_values_labels=df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum().loc[lambda df: df.values == 0].axes[0].tolist()
df_cleaned=df[X_numerical_not_missing_values_labels]
# df_cleaned = df_cleaned._get_numeric_data().drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea'],1)

# print(df_cleaned[df_cleaned._get_numeric_data().columns.tolist()].isnull().sum())

### Remove categorical features with missing values

In [15]:
# Checking for missing values in categorical values in the data set

# print(df[list(X_categorical_labels)].isnull().sum())

# Dropping categorical features with missing values
# The following instruction returns the list of features with no missing values

X_categorical_not_missing_values_labels = df[list(X_categorical_labels)].isnull().sum().loc[lambda df: df.values == 0].axes[0].tolist()
df_cleaned=df[X_categorical_not_missing_values_labels]
# df[list(X_categorical_labels)].drop(X_categorical_not_missing_values_labels, 1)


### Drop outliers in numerical values # WIP

In [16]:
def drop_outliers(df, field_name):
    distance = 1.5 * (np.percentile(df[field_name], 75) - np.percentile(df[field_name], 25))
    df.drop(df[df[field_name] > distance + np.percentile(df[field_name], 75)].index, inplace=True)
    df.drop(df[df[field_name] < np.percentile(df[field_name], 25) - distance].index, inplace=True)

# drop_outliers(df_cleaned, 'LotArea')    
df_cleaned_no_outlier = df.copy()
for feature in X_numerical_not_missing_values_labels:
    drop_outliers(df_cleaned_no_outlier, feature)    

In [17]:
print("nb of rows with outliers:{}, without: {}".format(df.shape[0],df_cleaned_no_outlier.shape[0]))


nb of rows with outliers:1460, without: 584


We are currently removing far too many items, to review later.
can be done with sklearn anyway http://scikit-learn.org/stable/modules/outlier_detection.html

# Feature Selection

In [18]:
# feature_columns=filter(lambda x: x !="SalePrice" and x!="Id", df.columns)
# feature_numerical_columns = df[feature_columns].select_dtypes(include=[np.number]).columns.tolist()
feature_numerical_columns=X_numerical_not_missing_values_labels
prediction_column="SalePrice"
# # print(feature_columns)
X = df[feature_numerical_columns]
Y=df[[prediction_column]];

In [19]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier();
model = model.fit(X, Y)    
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("label {} : {}".format(feature_numerical_columns[f],importances[indices[f]]))
# print(indices)

# plt.bar(range(X.shape[1]),importances[indices],align='center')
# plt.xticks(range(X.shape[1]),feature_numerical_columns,rotation=90);
# plt.rcParams["figure.figsize"] = (17,5)


/var/Applications/dataiku/code-envs/python/python3_6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


label MSSubClass : 0.10964960020166101
label LotArea : 0.05762900801533718
label OverallQual : 0.05641640508576516
label OverallCond : 0.0550864495724272
label YearBuilt : 0.052531858530184196
label YearRemodAdd : 0.05199151219333657
label BsmtFinSF1 : 0.05039290731018463
label BsmtFinSF2 : 0.0482381420005613
label BsmtUnfSF : 0.046735990607407095
label TotalBsmtSF : 0.04577890671613686
label 1stFlrSF : 0.044485977783274386
label 2ndFlrSF : 0.0372400210125505
label LowQualFinSF : 0.03508799439084796
label GrLivArea : 0.03424697843627001
label BsmtFullBath : 0.03030505103232351
label BsmtHalfBath : 0.028672644331549608
label FullBath : 0.026723221696382027
label HalfBath : 0.02605409105123498
label BedroomAbvGr : 0.02417062811785211
label KitchenAbvGr : 0.021144331007204902
label TotRmsAbvGrd : 0.01797817183279799
label Fireplaces : 0.01439393612528463
label GarageCars : 0.013443551752425006
label GarageArea : 0.012736261218064848
label WoodDeckSF : 0.010967144759041678
label OpenPorchS

To draw graph and select features

# Spot Check Algorithms
## Split dataset

In [26]:
from sklearn.model_selection import train_test_split

X = df[X_numerical_not_missing_values_labels]
Y=df[["SalePrice"]];
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.33,random_state=42)
print(Y_train.head())
X_train.head()

      SalePrice
615      137500
613      147000
1303     232000
486      156000
561      170000


,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
615,85,8800,6,7,1963,1963,763,0,173,936,...,120,0,0,0,0,0,0,5,2010,137500
613,20,8402,5,5,2007,2007,206,0,914,1120,...,0,30,0,0,0,0,0,12,2007,147000
1303,20,8688,7,5,2005,2005,0,0,1616,1616,...,208,59,0,0,0,0,0,4,2006,232000
486,20,10289,5,7,1965,1965,836,0,237,1073,...,0,0,0,0,0,0,0,6,2007,156000
561,20,10010,5,5,1974,1975,1071,123,195,1389,...,240,38,0,0,0,0,0,4,2006,170000


In [27]:

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score



In [ ]:

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, Y_train)

# # Make predictions using the testing set
# y_pre = regr.predict(X_test)
# print(classification_report(Y_test, y_pre))

In [1]:

#training
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier 

models = []
models.append(("SVC",SVC()))
models.append(("LinearSVC",LinearSVC()))
models.append(("KNeighbors",KNeighborsClassifier()))
models.append(("DecisionTree",DecisionTreeClassifier()))
models.append(("RandomForest",RandomForestClassifier()))

modelsWhichNeedNormalization = []
modelsWhichNeedNormalization.append(("LogisticRegression",LogisticRegression()))
modelsWhichNeedNormalization.append(("MLPClassifier",MLPClassifier(solver='lbfgs', random_state=0)))
modelsWhichNeedNormalization.append(("LinearRegression", LinearRegression()))
models

[('SVC', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)),
 ('LinearSVC',
  LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0)),
 ('KNeighbors',
  KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
             metric_params=None, n_jobs=1, n_neighbors=5, p=2,
             weights='uniform')),
 ('DecisionTree',
  DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, 

In [112]:
def trainOnModel(name, model, predictionColumn, X_train,X_test,Y_train,Y_test, isStandardizing=False, isVerbose=True):
    if isStandardizing:
        X_train = preprocessing.scale(X_train)
        X_test = preprocessing.scale(X_test)
    
    results = []
    names = []
    model = model.fit(X_train, Y_train)
        
    if isVerbose:
        print("Results on training set")
        print(classification_report(Y_train, model.predict(X_train)))
    
    y_pre = model.predict(X_test)
    
    if isVerbose:
        print("Results on dev set")
        print(classification_report(Y_test, y_pre))
    
    devScores=[]
    devScores.append(name)
    if predictionColumn=="Movement":
        devScores.append(f1_score(Y_test, y_pre,average='weighted'))
        devScores.extend(f1_score(Y_test, y_pre,average=None,labels=["STANDING","LYING_STRAIGHT","LYING_ON_SIDE","ROLLING"]))
    else:
        devScores.append(f1_score(Y_test, y_pre>0.5,average='binary'))
        #     print(devScores)
    
    for i in range(len(names)):
        print(names[i],results[i].mean())
        
    return y_pre, model, devScores

def trainOnMultipleModels(models,predictionColumn, X_train,X_test,Y_train,Y_test):
    results = []
    names = []
    allDevScores = []
    allModels = []
    for name,model in models:
        print("######################################")
        print("#### Training on {} ##################".format(name))
        y_pre, model, devScores=trainOnModel(name, model, predictionColumn, X_train,X_test,Y_train,Y_test)
        allDevScores.append(devScores)
        allModels.append(model)
    return allDevScores,allModels


def displayTrainOnEverythingBut1HorsePlot(modelDevScores):
    all=list(modelDevScores)
    modelNames=[(lambda row: row[0])(row) for row in all]
    names=[(lambda row: row[1])(row) for row in all]
    labels = ["GLOBAL","STANDING","LYING_STRAIGHT","LYING_ON_SIDE","ROLLING"]
    f1scores=[(lambda row: row[2])(row) for row in all]
    standingF1scores=[(lambda row: row[3])(row) for row in all]
    lyingStraightF1scores=[(lambda row: row[4])(row) for row in all]
    lyingOnSideF1scores=[(lambda row: row[5])(row) for row in all]
    rollingF1scores=[(lambda row: row[6])(row) for row in all]

    plt.rcParams["figure.figsize"] = (14,2)

    x = range(len(names))

    def createXaxis(start,end,extra):
        x=[]
        i=start
        while i<end:
            x.append(i+extra)
            i+=1
        return x

    ax = plt.subplot()
    plt.xticks(rotation=0)

    ax.bar(createXaxis(0,len(names),-0.2), f1scores,width=0.1,color='black',align='center')
    ax.bar(createXaxis(0,len(names),-0.1), standingF1scores,width=0.1,color='b',align='center')
    ax.bar(createXaxis(0,len(names),-0), lyingStraightF1scores,width=0.1,color='r',align='center')
    ax.bar(createXaxis(0,len(names),0.1), lyingOnSideF1scores,width=0.1,color='g',align='center')
    ax.bar(createXaxis(0,len(names),0.2), rollingF1scores,width=0.1,color='y',align='center')

    ax.set_xticks(x)
    ax.set_xticklabels(names)
    ax.grid('on', which='major', axis='y', color='black', linestyle='-', linewidth=0.5 )

    ax.legend(labels,bbox_to_anchor=(1,0.5), loc="center left")

    print(modelDevScores[0][0])
    plt.show()
    
